In [1]:
import os
from os import listdir
from os.path import isfile, join

import numpy as np
import torch
from torchvision import models
from pathlib import Path

In [2]:
TMP_PT = 'tmp.pt'

# to execute locally change path not in repo
IMAGENET_VAL_ROOT = '../../tmp/imgnet/val'
CUSTOM_COCO_ROOT = '../../tmp/cutsom-coco-data/images'

In [3]:
imagenet_images = [os.path.abspath(p) for p in Path(IMAGENET_VAL_ROOT).rglob('*.JPEG')]

In [4]:
coco_images = [os.path.abspath(p) for p in Path(CUSTOM_COCO_ROOT).rglob('*.jpg')]

In [5]:
imagenet_image_sizes = [os.path.getsize(f) for f in imagenet_images]
imagenet_image_sizes = np.array(imagenet_image_sizes)

In [6]:
coco_image_sizes = [os.path.getsize(f) for f in coco_images]
coco_image_sizes = np.array(coco_image_sizes)

In [7]:
print('IMAGENET')
print('Average image size in bytes: ')
avg_imagenet_image_size = np.average(imagenet_image_sizes)
print(avg_imagenet_image_size)
print('Median image size in bytes: ')
print(np.median(imagenet_image_sizes))

print('Average image size in kilobytes: ')
print(np.average(imagenet_image_sizes) / 1000)
print('Median image size in kilobytes: ')
print(np.median(imagenet_image_sizes) / 1000)


IMAGENET
Average image size in bytes: 
134131.20638
Median image size in bytes: 
126689.0
Average image size in kilobytes: 
134.13120637999998
Median image size in kilobytes: 
126.689


In [8]:
print('CUSTOM COCO')
print('Average image size in bytes: ')
avg_coco_image_size = np.average(coco_image_sizes)
print(avg_coco_image_size)
print('Median image size in bytes: ')
print(np.median(coco_image_sizes))

print('Average image size in kilobytes: ')
print(np.average(coco_image_sizes) / 1000)
print('Median image size in kilobytes: ')
print(np.median(coco_image_sizes) / 1000)

CUSTOM COCO
Average image size in bytes: 
168029.00979216627
Median image size in bytes: 
159030.5
Average image size in kilobytes: 
168.02900979216628
Median image size in kilobytes: 
159.0305


In [9]:
models = [models.mobilenet_v2, models.googlenet, models.resnet18, models.resnet50, models.resnet152]
model_names = [m.__name__ for m in models]

In [10]:
state_dict_sizes = []

for model in models:
    tmp_model = model(pretrained=True)
    torch.save(tmp_model.state_dict(), TMP_PT)
    state_dict_size = os.path.getsize(TMP_PT)
    state_dict_sizes.append(state_dict_size)
    print('{} \t state_dict size: \t {} megabyte'.format(model.__name__, state_dict_size / 10 ** 6))
    os.remove(TMP_PT)


mobilenet_v2 	 state_dict size: 	 14.266675 megabyte
googlenet 	 state_dict size: 	 26.681183 megabyte
resnet18 	 state_dict size: 	 46.837875 megabyte
resnet50 	 state_dict size: 	 102.548719 megabyte
resnet152 	 state_dict size: 	 241.692183 megabyte


In [11]:
print("IMAGENET: Uncompressed images")
for model_name, dict_size in zip(model_names, state_dict_sizes):
    per_state_dict = dict_size / avg_imagenet_image_size
    print('{} \t images instead of state_dict: \t {}'.format(model_name, per_state_dict))

IMAGENET: Uncompressed images
mobilenet_v2 	 images instead of state_dict: 	 106.36357776117991
googlenet 	 images instead of state_dict: 	 198.91853447147085
resnet18 	 images instead of state_dict: 	 349.1944661058673
resnet50 	 images instead of state_dict: 	 764.5403464833879
resnet152 	 images instead of state_dict: 	 1801.9086648283378


In [12]:
print("COCO: Uncompressed images")
for model_name, dict_size in zip(model_names, state_dict_sizes):
    per_state_dict = dict_size / avg_coco_image_size
    print('{} \t images instead of state_dict: \t {}'.format(model_name, per_state_dict))




COCO: Uncompressed images
mobilenet_v2 	 images instead of state_dict: 	 84.90602317805917
googlenet 	 images instead of state_dict: 	 158.789146189707
resnet18 	 images instead of state_dict: 	 278.74874141038737
resnet50 	 images instead of state_dict: 	 610.3036560582109
resnet152 	 images instead of state_dict: 	 1438.3955681161672
